In [5]:
import gymnasium as gym
import numpy as np

In [6]:
env = gym.make('CartPole-v1', render_mode='human')
state = env.reset()[0]
done = False

while not done:
    env.render()
    action = np.random.choice([0, 1])
    next_state, reward, terminated, truncated, info = env.step(action)
    done = terminated | truncated
env.close()

: 